In [ ]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [ ]:
!pip install optuna

In [ ]:
# Primitive Data Processing

from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler




# Check for null values
print("Missing values in train_df:")
print(train_df.isnull().sum())

print("\nMissing values in test_df:")
print(test_df.isnull().sum())



print("\nPrimitive processing steps checked.")

In [ ]:
# Examine categorical feature distributions

categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']

# Determine the number of rows and columns for the grid
n_features = len(categorical_features)
n_cols = 3  # You can adjust the number of columns as needed
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

for i, feature in enumerate(categorical_features):
    sns.countplot(data=train_df, x=feature, order=train_df[feature].value_counts().index, ax=axes[i])
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
train_df.head()

In [ ]:
train_df=train_df.drop("id",axis='columns')
test_df=test_df.drop("id",axis='columns')

In [ ]:
train_df.head()

In [ ]:
x=train_df.drop("WeightCategory",axis='columns')
y=train_df['WeightCategory']

In [ ]:
numeric_features = ['Age', 'Height','Weight','FCVC','NCP','CH2O','FAF','TUE']

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df[numerical_features].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('t1',OneHotEncoder(drop='first'),['CAEC']),
    ('t2',OneHotEncoder(drop='first'),['CALC']),
    ('t3', StandardScaler(), numeric_features),

    ('t4',OneHotEncoder(drop='first'),['Gender','family_history_with_overweight','FAVC','SMOKE','SCC','MTRANS'])
],remainder='passthrough')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Import pandas to ensure train_df is accessible

# Assuming 'y' is the target variable (WeightCategory)
# If y is a pandas Series, we can directly use it, otherwise convert it
if not isinstance(y, pd.Series):
  y_series = pd.Series(y, name="WeightCategory")
else:
  y_series = y

plt.figure(figsize=(10, 6)) # Adjusted figure size for better readability
sns.countplot(data=None, x=y_series, order=y_series.value_counts().index) # Use data=None and pass the series directly, order by count
plt.title("Class Distribution of Obesity Categories")
plt.xlabel("Obesity Class")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right') # Rotate labels for better readability if many classes
plt.tight_layout()
plt.show()

In [ ]:
numerical_features = ['Age', 'Height','Weight','FCVC','NCP','CH2O','FAF','TUE']

train_df[numerical_features].hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
target = 'WeightCategory'

n_features = len(categorical_features)
n_cols = 2
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 6)) # Adjust figsize as needed
axes = axes.flatten()
for i, feature in enumerate(categorical_features):
    sns.countplot(data=train_df, x=feature, hue=target, palette='viridis', ax=axes[i])
    axes[i].set_title(f'Relationship between {feature} and {target}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].legend(title=target)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

train_df = pd.read_csv('train.csv')

plt.figure(figsize=(10, 6))
sns.countplot(data=train_df, x='WeightCategory', order=train_df['WeightCategory'].value_counts().index)
plt.title('Distribution of WeightCategory')
plt.xlabel('Weight Category')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
y = np.array(y)
y.shape

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

encoder = OrdinalEncoder()
y_encoded = encoder.fit_transform(np.array(y).reshape(-1, 1))
y_encoded.shape

In [ ]:
from xgboost import XGBClassifier

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 460, 480),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.065),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 5),
        'gamma': trial.suggest_float('gamma', 0.2, 0.4),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.4, 0.6),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 0.5),
    }

    model = XGBClassifier(**params, objective='multi:softmax')

    # cross-validation
    score = cross_val_score(model, x_transformed, y_encoded, cv=3, scoring='accuracy').mean() # Use original y

    return score


In [ ]:
x_transformed = transformer.fit_transform(x)

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


study = optuna.create_study(direction='maximize')

# Encode y to int
label_encoder = LabelEncoder()
y_encoded_labels = label_encoder.fit_transform(y)

# 100 trials optimization - 50 and 25 didnt yeild the same performance
study.optimize(lambda trial: objective(trial), n_trials=100, show_progress_bar=True)

# Print the best hyperparameters and the best score

print("Best hyperparameters:", study.best_params)
print("Best accuracy:", study.best_value)
# best_params={'n_estimators': 476,  'learning_rate': 0.056175675016670416, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.3354617624351886, 'subsample': 0.7871797069127701, 'colsample_bytree': 0.5071468557602662, 'reg_alpha': 0.5393018307093757, 'reg_lambda': 0.0809287158560023}
final_model = XGBClassifier(**study.best_params, random_state=42, objective='multi:softmax', num_class=len(label_encoder.classes_))
final_model.fit(x_transformed, y_encoded_labels)

# Make predictions on the training data to evaluate accuracy
y_pred = final_model.predict(x_transformed)

# Calculate and print the accuracy
accuracy = accuracy_score(y_encoded_labels, y_pred)
print(f"Accuracy on the training data: {accuracy}")



In [ ]:

# fit transform predict
x_transformed = transformer.fit_transform(x)
test_transformed = transformer.transform(test_df)
test_predictions_encoded = final_model.predict(test_transformed)

# decode
test_predictions = encoder.inverse_transform(test_predictions_encoded.reshape(-1, 1))

# submission
submission_df = pd.DataFrame({'id': test_df.index + 15533, 'Weight_Category': test_predictions.flatten()})
submission_df.to_csv('xgb_optuna.csv', index=False)

print("xgb_optuna.csv created successfully with predictions from the optimized model!")

In [ ]:
numerical_features = ['Age', 'Height','Weight','FCVC','NCP','CH2O','FAF','TUE']
target = 'WeightCategory'

for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=train_df, x=target, y=feature, order=train_df[target].value_counts().index)
    plt.title(f'{feature} vs {target}')
    plt.xlabel(target)
    plt.ylabel(feature)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()